In [6]:
import polars as pl

from rs_graph.db import constants as db_constants

###############################################################################


def _read_table(table: str) -> pl.DataFrame:
    return pl.read_database_uri(
        f"SELECT * FROM {table}",
        f"sqlite:///{db_constants.V2_DATABASE_PATHS.dev}",
    )


def load_pairs() -> pl.DataFrame:
    # Read all the tables we need
    docs = _read_table("document")
    repos = _read_table("repository")
    pairs = _read_table("document_repository_link")

    repo_contribs = _read_table("repository_contributor")

    # Drop to unique doc and unique repo in pairs
    pairs = pairs.unique(
        subset="document_id",
        keep="none",
    ).unique(
        subset="repository_id",
        keep="none",
    )

    # Join the tables to get the positive examples
    df = (
        pairs.select(
            "document_id",
            "repository_id",
        )
        .join(
            docs.select(
                pl.col("id").alias("document_id"),
                "publication_date",
            ),
            on="document_id",
            how="left",
        )
        .join(
            repos.select(
                pl.col("id").alias("repository_id"),
                "creation_datetime",
                "last_pushed_datetime",
            ),
            on="repository_id",
            how="left",
        )
    )

    return df, repo_contribs


df, repo_contribs = load_pairs()
df

document_id,repository_id,publication_date,creation_datetime,last_pushed_datetime
i64,i64,date,datetime[ns],datetime[ns]
1,1,2015-12-01,2014-05-05 21:14:57,2018-04-29 16:11:15
2,2,2022-10-01,2021-11-09 04:35:52,2021-11-23 04:14:31
3,3,2017-12-08,2017-07-10 14:22:18,2018-07-11 12:15:35
4,4,2020-05-18,2019-04-01 16:16:34,2021-07-12 10:28:10
5,5,2017-09-29,2016-12-13 22:27:50,2024-03-25 19:35:00
…,…,…,…,…
138590,132078,2021-01-01,2021-02-06 13:04:58,2022-07-20 11:14:06
138591,132080,2021-01-01,2021-04-27 09:05:06,2022-01-05 21:05:54
138592,132081,2020-01-01,2020-05-25 19:59:39,2022-06-13 17:34:41


In [8]:
repo_contribs.group_by("repository_id").agg(pl.len().alias("num_contributors"))[
    "num_contributors"
].describe()

statistic,value
str,f64
"""count""",128992.0
"""null_count""",0.0
"""mean""",2.28139
"""std""",3.92659
"""min""",1.0
"""25%""",1.0
"""50%""",1.0
"""75%""",2.0
"""max""",32.0


In [2]:
# Create columns for difference between publication date and creation date
# and publication date and last pushed date
df = df.with_columns(
    (pl.col("publication_date") - pl.col("creation_datetime"))
    .dt.total_days()
    .abs()
    .alias("publication_date_creation_date_diff"),
    (pl.col("publication_date") - pl.col("last_pushed_datetime"))
    .dt.total_days()
    .abs()
    .alias("publication_date_last_pushed_date_diff"),
)

# Also create a "years" column for both of these
df = df.with_columns(
    (pl.col("publication_date_creation_date_diff") / 365.25).alias(
        "publication_date_creation_date_diff_years"
    ),
    (pl.col("publication_date_last_pushed_date_diff") / 365.25).alias(
        "publication_date_last_pushed_date_diff_years"
    ),
)

df["publication_date_creation_date_diff"].describe(
    percentiles=[0.5, 0.75, 0.9, 0.95, 0.99],
).filter(pl.col("statistic").is_in(["mean", "std", "50%", "75%", "90%", "95%", "99%", "max"]))

statistic,value
str,f64
"""mean""",248.549657
"""std""",343.716394
"""50%""",162.0
"""75%""",299.0
"""90%""",520.0
"""95%""",792.0
"""99%""",1790.0
"""max""",8280.0


In [3]:
df["publication_date_last_pushed_date_diff"].describe(
    percentiles=[0.5, 0.75, 0.9, 0.95, 0.99],
).filter(pl.col("statistic").is_in(["mean", "std", "50%", "75%", "90%", "95%", "99%", "max"]))

statistic,value
str,f64
"""mean""",494.791609
"""std""",513.288314
"""50%""",335.0
"""75%""",676.0
"""90%""",1153.0
"""95%""",1543.0
"""99%""",2391.0
"""max""",8893.0


In [4]:
# Same for years
df["publication_date_creation_date_diff_years"].describe(
    percentiles=[0.5, 0.75, 0.9, 0.95, 0.99],
).filter(pl.col("statistic").is_in(["mean", "std", "50%", "75%", "90%", "95%", "99%", "max"]))

statistic,value
str,f64
"""mean""",0.680492
"""std""",0.941044
"""50%""",0.443532
"""75%""",0.818617
"""90%""",1.423682
"""95%""",2.168378
"""99%""",4.900753
"""max""",22.669405


In [5]:
df["publication_date_last_pushed_date_diff_years"].describe(
    percentiles=[0.5, 0.75, 0.9, 0.95, 0.99],
).filter(pl.col("statistic").is_in(["mean", "std", "50%", "75%", "90%", "95%", "99%", "max"]))

statistic,value
str,f64
"""mean""",1.354666
"""std""",1.405307
"""50%""",0.91718
"""75%""",1.850787
"""90%""",3.156742
"""95%""",4.224504
"""99%""",6.546201
"""max""",24.347707
